In [ ]:
!pip install -U  wandb transformers optuna plotly==4.14.3 "notebook==5.3" "ipywidgets==7.5"

  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
  Found existing installation: notebook 5.3.1
    Uninstalling notebook-5.3.1:
      Successfully uninstalled notebook-5.3.1
  Found existing installation: ipywidgets 7.6.3
    Uninstalling ipywidgets-7.6.3:
      Successfully uninstalled ipywidgets-7.6.3


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('./drive', force_remount=True)

Mounted at ./drive


In [ ]:
dir = '/content/drive/MyDrive/BiasCorp/bias_corp_racial.csv'

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.nn.functional as F
import wandb
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, r2_score, average_precision_score, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer
from collections import namedtuple
from dataclasses import dataclass
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from pandas.plotting import parallel_coordinates
import plotly
import plotly.figure_factory as ff
import plotly.offline as py
import math
from scipy.stats import zscore, norm
from scipy.special import ndtr
py.init_notebook_mode(connected=True)
from IPython.display import HTML
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

import copy
# from datasets import  load_metric
from transformers import BertModel, BertTokenizerFast, BertLayer
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from nltk.tokenize import TweetTokenizer

%matplotlib inline
plt.set_cmap('jet')

<Figure size 432x288 with 0 Axes>

In [ ]:
wandb.login(key="")
wandb.init(project="final-paper", id='my', save_code=True, name='temp', reinit=True, entity="walexi4great")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH='all'

env: WANDB_LOG_MODEL=true
env: WANDB_WATCH='all'


In [ ]:
BATCH_SIZE = 8
NUM_WORKERS = 3
MAX_LEN = 512
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [ ]:
df = pd.read_csv(dir)

In [ ]:
df.head(3)

,text,bias_1,bias_2,bias_3,confidence_1,confidence_2,confidence_3,time_1,time_2,time_3,sentiment_score,source
0,"'' Democrats needed someone like Obama, half w...",4,3,2,4,3,5,82,94,295,0.205548,BB
1,'Greatest threat to the Black Famiky is the wh...,1,3,4,6,6,4,48,37,23,-0.189737,BB
2,"'In March, Black Lives Matter activists marche...",3,3,2,7,7,7,140,2495,75,-0.282843,BB


In [ ]:
df['confidence_sum'] = df.apply(lambda row: sum([row['confidence_1'], row['confidence_2'], row['confidence_3']]), axis=1)

In [ ]:
df['class_indices'] = df.apply(lambda row: np.asarray([row['bias_1'], row['bias_2'], row['bias_3']]), axis=1)

In [ ]:
# df.loc[:, 'confidence_2'] = df.confidence_2.apply(lambda ele: ele/10)
# df.loc[:, 'confidence_3'] = df.confidence_3.apply(lambda ele: ele/10)
df['conf_scores'] = df.apply(lambda row: np.asarray([row['confidence_1'], row['confidence_2'], row['confidence_3']])/row['confidence_sum'], axis=1)

In [ ]:
@dataclass
class Data:
  train: pd.DataFrame
  val: pd.DataFrame
  test: pd.DataFrame

In [ ]:
def shuffle_dist(dataframe: pd.DataFrame, split_ratio: float=0.6) -> Data:
  """randomly split dataset into train and val set based on the source column
  Parameters
  ----------
  dataframe : pd.Dataframe
    dataset in pandas dataframe type
  split_ratio : float
    for a given source, percent of entries to split

  Returns
  -------
  Data
    dataclass with fields train, val, and test

  """
  coin = random.choice([0,1])
  if coin:
    ratio = int(len(dataframe[dataframe['source']=='Fox']) * split_ratio)
    train = pd.concat([ dataframe[dataframe['source']=='BB'], dataframe[dataframe['source']=='Fox'][:ratio] ])
    val = dataframe[dataframe['source']=='Fox'][ratio:]
  else:
    ratio = int(len(dataframe[dataframe['source']=='BB']) * split_ratio)
    train = pd.concat([ dataframe[dataframe['source']=='Fox'], dataframe[dataframe['source']=='BB'][:ratio] ])
    val = dataframe[dataframe['source']=='BB'][ratio:]
  test = dataframe[dataframe['source']=='Youtube']

  return Data(train,val,test)


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
import re

regex = re.compile("(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)|(\\n)|(\\r)|(\\t)|(\<+?.+?\>)", re.IGNORECASE)
regex2 = re.compile("(\\')")

In [ ]:
candidates = []
matches = []
matches2 = []
for text in df.text:
  if len(tokenizer.tokenize(text))>512:
    candidates.append(text)
  if regex.match(text):
    matches.append(text)
  if regex2.match(text):
    matches2.append(text)
  MAX_LEN = max(MAX_LEN, len(tokenizer.tokenize(text)))


Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
MAX_LEN, len(candidates), len(matches), len(matches2)

(623, 1, 815, 5)

In [ ]:
df.loc[:, 'text'] = df.text.apply(lambda ele: regex.sub("",ele))
df.loc[:, 'text'] = df.text.apply(lambda ele: regex2.sub("'",ele))

In [ ]:
MAX_LEN = 0
candidates = []
matches = []
matches2 = []
for text in df.text:
  if len(tokenizer.tokenize(text))>512:
    candidates.append(text)
  if regex.match(text):
    matches.append(text)
  if regex2.match(text):
    matches2.append(text)
  MAX_LEN = max(MAX_LEN, len(tokenizer.tokenize(text)))

In [ ]:
MAX_LEN, len(candidates), len(matches), len(matches2)

(478, 0, 0, 24)

In [ ]:
matches2

## Dataset

In [ ]:
df.head(2)

,text,bias_1,bias_2,bias_3,confidence_1,confidence_2,confidence_3,time_1,time_2,time_3,sentiment_score,source,confidence_sum,class_indices,conf_scores
0,"'' Democrats needed someone like Obama, half w...",4,3,2,4,3,5,82,94,295,0.205548,BB,12,"[4, 3, 2]","[0.3333333333333333, 0.25, 0.4166666666666667]"
1,'Greatest threat to the Black Famiky is the wh...,1,3,4,6,6,4,48,37,23,-0.189737,BB,16,"[1, 3, 4]","[0.375, 0.375, 0.25]"


In [ ]:
def one_hot(conf_scores, cls_indices):
  empty = torch.zeros((1,6), dtype=torch.float)
  cls = torch.tensor([cls_indices], dtype=torch.long)
  conf = torch.tensor([conf_scores], dtype=torch.float)

  return empty.scatter_(1, cls, conf, reduce='add')


In [ ]:
class BiasDataset(Dataset):
  def __init__(self, df, tokenizer, max_len):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.config = {
        'max_len': max_len,
        'return_attention_mask': True,
        'padding': 'max_length',
        'truncation': True,
        'return_tensors': 'pt',
        'return_token_type_ids': False
    }

  def __len__(self):
    return len(self.df)

  def __getitem__(self, item):
    entry = self.df.iloc[item]
    text = entry.text
    config = {
        'max_length': self.max_len,
        'return_attention_mask': True,
        'padding': 'max_length',
        'truncation': True,
        'return_tensors': 'pt',
        'return_token_type_ids': False
    }
    encoding = self.tokenizer(text, **config)

    try:
      conf_scores = entry.conf_scores
      indices = entry.class_indices
      target = one_hot(conf_scores, indices)

      return {
          'text': text,
          'input_ids': encoding['input_ids'],
          'attention_mask' : encoding['attention_mask'],
          'target': target
      }
    except KeyError:
      return {
          'text': text,
          'input_ids': encoding['input_ids'],
          'attention_mask': encoding['attention_mask']
      }

In [ ]:
dataset = shuffle_dist(df)

In [ ]:
train_ds = BiasDataset(dataset.train, max_len=MAX_LEN, tokenizer=tokenizer)
val_ds = BiasDataset(dataset.val, max_len=MAX_LEN, tokenizer=tokenizer)
test_ds  = BiasDataset(dataset.test, max_len=MAX_LEN, tokenizer=tokenizer)

In [ ]:
train_dl = DataLoader(train_ds, num_workers=NUM_WORKERS, batch_size= BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
dataloader = Data(train_dl, val_dl, test_dl)

In [ ]:
class BackBone(nn.Module):

  def __init__(self, trials=None):

    super(BackBone, self).__init__()

    bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

    for layer in bert.parameters():
      layer.requires_grad = False

    self.bert = bert

    # hopfield = EncoderLayer(bert.config,0.2)

    # self.bert = replaceLayer(bert, hopfield, [7,8,9])
    # # self.bi_lstm = return sequences and apply dropout
    # # max_pool
    # # dense then dropout and dense_output
    # self.hopfield_pool = HopfieldPooling(self.bert.config.hidden_size, hidden_size=32, output_size=self.bert.config.hidden_size, num_heads=1)
    if(trials):
      self.dropout = nn.Dropout(trials.suggest_uniform('dropout_rate', 0.2, 0.5))
    else:
      self.dropout = nn.Dropout(0.3)

    self.lin = nn.Linear(self.bert.config.hidden_size, 6)

  def forward(self, input_ids, attention_mask = None):
    output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_attentions=False
        )

    out = output.pooler_output

    # out = out.unsqueeze(1)

    # out = self.hopfield_pool(out)

    out = self.lin(self.dropout(out))

    return out, output.attentions


In [ ]:
class Trainer:
  def __init__(self, model, dataloader, batch_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.model  = model.to(self.device)
    self.dataloader = dataloader #move to data prep to init method later
    self.batch_size = batch_size
    bce_loss = nn.BCEWithLogitsLoss()
    self.bce_loss = bce_loss
    self.label_binarizer = MultiLabelBinarizer(classes=[0,1,2,3,4,5])
    wandb.watch(self.model, bce_loss, log='all')

# for each batch, return loss, r2_score,
  def compute_metrics(self, logits, targets, end_of_epoch=False, isTrain=True):
    if end_of_epoch and not isTrain:

      # transform to label indicators for each topk
      pred_matrix = [self.label_binarizer.fit_transform(logits[i]) for i in range(3)]
      targ_matrix = [self.label_binarizer.fit_transform(targets[i]) for i in range(3)]

      # ap for each class in an array of class length
      average_precision = [ average_precision_score(targ_matrix[i], pred_matrix[i], average=None) for i in range(3) ]
      # print(pred_matrix[0].shape) (8392, 2)
      # mean average precision
      map = [average_precision_score(targ_matrix[i], pred_matrix[i], average='macro') for i in range(3)]

      # array of tuple(precision recall f1_score) for each class
      prec_recall_f1 = [ [ (precision_recall_fscore_support(targ_matrix[j][:,i], pred_matrix[j][:,i], average='micro'))[0:3] for i in range(6)] for j in range(3)]

      return average_precision, map, prec_recall_f1



    loss = self.bce_loss(logits, targets)

    if not isTrain:
      prob = F.softmax(logits, dim=1)
      prob = prob.cpu()
    # # r2_score = r2_score(targets.numpy(), prob, multioutput='raw_values')
      topk_preds = [0]*3
      topk_targs = [0]*3
      for i in range(3):
        topk_pred = prob.topk(i+1).indices.numpy()
        topk_targ = targets.cpu().topk(i+1).indices.numpy()
        topk_preds[i]=topk_pred
        topk_targs[i]=topk_targ

      return loss, topk_preds, topk_targs

    return loss

  def fit(self, epochs, lr):
    config = wandb.config
    config.batch_size = self.batch_size
    config.epochs = epochs
    config.lr = lr
    config.no_cuda = False
    config.log_interval = 10

    optimizer = AdamW(self.model.parameters(), lr=config.lr, correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(self.dataloader.train)* epochs)



    for epoch in tqdm(range(config.epochs)):
      train_loss  = self.train(optimizer)
      val_loss, avg_prec, map, prec_rec_f1 = self.val()
      # self.test() # may run after x epochs given the best model so far

      wandb.log({'train_loss_epoch': np.mean(train_loss), 'epoch': epoch},commit=False)
      wandb.log(
          {'val_loss_epoch': np.mean(val_loss),
           'average_precision': avg_prec,
           'mean_avg_precision': map,
           'precision_recall_f1': prec_rec_f1,
           'epoch': epoch
          })




  def train(self, optimizer):

    self.model.train()
    total_loss = []
    train_loader = self.dataloader.train


    for batch_idx, entry in enumerate(train_loader):

      input_ids = entry['input_ids'].to(self.device)
      attention_mask = entry['attention_mask'].to(self.device)
      targets = entry['target'].to(self.device)


      optimizer.zero_grad()

      logits, _ = self.model(input_ids.squeeze(1), attention_mask.squeeze(1))

      loss = self.compute_metrics(logits, targets.squeeze(1))
      loss.backward()

      optimizer.step()

      if ((batch_idx+1)%25==0):
        wandb.log({'train_loss_batch': loss.item()})


      total_loss.append(loss.item())

    return total_loss


  def val(self):

    self.model.eval()

    total_loss = []
    topk_predictions = None
    topk_targets = None
    val_loader = self.dataloader.val

    with torch.no_grad():

      for entry in val_loader:

        input_ids = entry['input_ids'].to(self.device)
        attention_mask = entry['attention_mask'].to(self.device)
        targets = entry['target'].to(self.device)

        logits, _ = self.model(input_ids.squeeze(1), attention_mask.squeeze(1))
        # k:int -> arr

        loss, topk_preds, topk_targs = self.compute_metrics(logits, targets.squeeze(1), isTrain=False)

        if topk_predictions is None:
          topk_predictions = topk_preds
        else:
          topk_predictions = [np.concatenate((prev, curr), axis=0) for prev, curr in zip(topk_predictions,topk_preds)]

        if topk_targets is None:
          topk_targets = topk_targs
        else:
          topk_targets = [np.concatenate((prev, curr), axis=0) for prev, curr in zip(topk_targets,topk_targs)]

        total_loss.append(loss.item())

      avg_prec, map, prec_rec_f1 = self.compute_metrics(topk_predictions, topk_targets, end_of_epoch=True, isTrain=False)

    return total_loss, avg_prec, map, prec_rec_f1

  def test(self):
    # self.model.eval()
    pass

In [ ]:
model = BackBone()

In [ ]:
trainer = Trainer(model, dataloader, BATCH_SIZE)

In [ ]:
trainer.fit(1, 0.0005)

In [ ]:
def objective(trial):


## PREV

In [ ]:
# fig = df.groupby(['source','bias']).count()['text'].loc['BB'].plot(kind='bar')
# HTML(fig.to_html())
fig = df['source'].plot(kind='hist', histnorm='probability', labels=dict(value='source'))
HTML(fig.to_html())

In [ ]:
def plot(df, x_input, y_input):
  x = df[x_input]
  y = df[y_input]

  fig = go.Figure()
  fig.add_trace(go.Histogram2dContour(
          x = x,
          y = y,
          histnorm='probability',
          colorscale = 'Blues',
          reversescale = True,
          xaxis = 'x',
          yaxis = 'y'
      ))
  fig.add_trace(go.Scatter(
          x = x,
          y = y,
          xaxis = 'x',
          yaxis = 'y',
          mode = 'markers',
          marker = dict(
              color = 'rgba(0,0,0,0.3)',
              size = 3
          )
      ))
  fig.add_trace(go.Histogram(
          y = y,
          xaxis = 'x2',
          histnorm='probability',
          marker = dict(
              color = 'rgba(0,0,0,1)'
          )
      ))
  fig.add_trace(go.Histogram(
          x = x,
          yaxis = 'y2',
          histnorm='probability',
          marker = dict(
              color = 'rgba(0,0,0,1)'
          )
      ))

  fig.update_layout(
      # autosize = False,
      xaxis_title=x_input,
      yaxis_title=y_input,
      xaxis = dict(
          zeroline = False,
          domain = [0,0.85],
          showgrid = False
      ),
      yaxis = dict(
          zeroline = False,
          domain = [0,0.85],
          showgrid = False
      ),
      xaxis2 = dict(
          zeroline = False,
          domain = [0.85,1],
          showgrid = False
      ),
      yaxis2 = dict(
          zeroline = False,
          domain = [0.85,1],
          showgrid = False
      ),
      height = 600,
      width = 600,
      bargap = 0,
      hovermode = 'closest',
      showlegend = False
  )
  return fig


In [ ]:
benchmk_bias = 2 #using 0.8 of scale http://portal.acm.org/citation.cfm?id=175282
benchmk_conf = 7
ep = 10e-6 # for numerical stability
alpha = 0.05
# z = (becn - mean)/std

In [ ]:
fig = plot(df, 'bias_1','confidence_1')
HTML(fig.to_html())

In [ ]:
fig = plot(df, 'bias_2','confidence_2')
HTML(fig.to_html())

In [ ]:
fig = plot('bias_3','confidence_3')
HTML(fig.to_html())

In [ ]:
sns.displot(df['sentiment_score'])
plt.show()

In [ ]:
sns.displot(df['sentiment_score'])
plt.show()

In [ ]:
df['bias_std'] = df.apply(lambda row: np.std([row['bias_1'], row['bias_2'], row['bias_3']]), axis=1)
df['confidence_std'] = df.apply(lambda row: np.std([row['confidence_1'], row['confidence_2'], row['confidence_3']]), axis=1)

In [ ]:
# df['bias_avg'] = df.apply(lambda row: np.mean([row['bias_1'], row['bias_2'], row['bias_3']]), axis=1)
# df['bias_z'] = df.apply(lambda row: (row['bias_avg']-row['bias_z'])/row['bias_std']), axis=1)
# df['bias_z'] = df.apply(lambda row: (row['bias_avg'] - benchmk_bias)/(row['bias_std']+ep), axis=1)
# df['bias_p'] = df.apply(lambda row: round(ndtr(row['bias_z']))), axis=1)
# df['bias_p'] = df.apply(lambda row: 1-norm.sf(row['bias_z']), axis=1)
# df['bias_cv'] = df.apply(lambda row: row['bias_std']/(row['bias_avg']+ep), axis=1) #coefficient of variation
# df['confidence_avg'] = df.apply(lambda row: np.mean([row['confidence_1'], row['confidence_2'], row['confidence_3']]), axis=1)
# df['confidence_z'] = df.apply(lambda row: (row['confidence_avg'] - benchmk_conf)/(row['confidence_std']+ep), axis=1)
# df['confidence_p'] = df.apply(lambda row: 1-norm.sf(row['confidence_z']), axis=1)
# df['confidence_cv'] = df.apply(lambda row: row['confidence_std']/(row['confidence_avg']+ep), axis=1) #coefficient of variation

In [ ]:
df['bias_avg'] = df.apply(lambda row: np.mean([row['bias_1'], row['bias_2'], row['bias_3']]), axis=1)
df['confidence_avg'] = df.apply(lambda row: np.mean([row['confidence_1'], row['confidence_2'], row['confidence_3']]), axis=1)

In [ ]:
fig = plot('bias_avg','confidence_avg')
HTML(fig.to_html())

In [ ]:
df['bias_cv'] = df.apply(lambda row: row['bias_std']/(row['bias_avg']+ep), axis=1) #coefficient of variation
df['confidence_cv'] = df.apply(lambda row: row['confidence_std']/(row['confidence_avg']), axis=1) #coefficient of variation

In [ ]:
sns.histplot(df['confidence_std'],stat='density', kde=True)
plt.show()

In [ ]:
sns.histplot(df['confidence_std'],stat='probability', kde=True)
plt.show()

In [ ]:
df.describe()

In [ ]:
fig_go = go.Figure()
fig_go.add_trace(go.Violin(y=df['confidence_1'], x=df['bias_1'], name='resp1'))
fig_go.add_trace(go.Violin(y=df['confidence_2'], x=df['bias_2'], name='resp2'))
fig_go.add_trace(go.Violin(y=df['confidence_3'], x=df['bias_3'], name='resp3'))
# fig_go.update_layout(yaxis_title='normalized mositure', boxmode='group')
fig_go.update_traces(box_visible=True, meanline_visible=True)
fig_go.update_layout(violinmode='group', yaxis_title='Confidence Score', xaxis_title='Bias Level')
HTML(fig_go.to_html())

In [ ]:
sns.histplot(df['bias_avg'],stat='density', kde=True)
plt.show()

In [ ]:
sns.histplot(df['confidence_avg'],stat='density', kde=True)
plt.show()

In [ ]:
sns.histplot(df['bias_cv'],stat='density', kde=True)
plt.show() #low cv means lower variability

In [ ]:
sns.histplot(df['confidence_cv'],stat='density', kde=True)
plt.show() #low cv means lower variability

In [ ]:
fig = plot('bias_cv','confidence_cv')
HTML(fig.to_html())

In [ ]:
len(df[df['bias_cv']<0.35])/len(df)

In [ ]:
len(df[df['bias_cv']<0.3])/len(df)

In [ ]:
len(df[df['bias_cv']<0.4])/len(df)

In [ ]:
df_4 = df[df['confidence_cv']<0.4]
df_35 = df[df['confidence_cv']<0.35]
df_3 = df[df['confidence_cv']<0.3]
df_2 = df[df['confidence_cv']<0.2]
df_1 = df[df['confidence_cv']<0.1]

In [ ]:
len(df_4)/len(df), len(df_35)/len(df), len(df_3)/len(df), len(df_2)/len(df), len(df_1)/len(df)

In [ ]:
df_4['weighted_bias'] = df.apply(lambda row: np.average([row['bias_1'],row['bias_2'],row['bias_3']],weights=[row['confidence_1'],row['confidence_1'],row['confidence_1']]), axis=1)
df_35['weighted_bias'] = df.apply(lambda row: np.average([row['bias_1'],row['bias_2'],row['bias_3']],weights=[row['confidence_1'],row['confidence_1'],row['confidence_1']]), axis=1)
df_3['weighted_bias'] = df.apply(lambda row: np.average([row['bias_1'],row['bias_2'],row['bias_3']],weights=[row['confidence_1'],row['confidence_1'],row['confidence_1']]), axis=1)
df_2['weighted_bias'] = df.apply(lambda row: np.average([row['bias_1'],row['bias_2'],row['bias_3']],weights=[row['confidence_1'],row['confidence_1'],row['confidence_1']]), axis=1)
df_1['weighted_bias'] = df.apply(lambda row: np.average([row['bias_1'],row['bias_2'],row['bias_3']],weights=[row['confidence_1'],row['confidence_1'],row['confidence_1']]), axis=1)

In [ ]:
df_4.drop(columns=['time_1','time_2','time_3']).describe()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df_4['weighted_bias'],histnorm='probability', cumulative_enabled=True)])
fig.update_layout(yaxis_title='Cummulated Probability', xaxis_title='Weighted Bias')
HTML(fig.to_html())

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df_4['weighted_bias'],histnorm='probability', cumulative_enabled=False)])
fig.update_layout(yaxis_title='Probability', xaxis_title='Weighted Bias')
HTML(fig.to_html())

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df_3['weighted_bias'], histnorm='probability', cumulative_enabled=True)])

HTML(fig.to_html())

In [ ]:
len(df_new), len(df)

In [ ]:
len(df_new)/len(df)

In [ ]:
# sns.FacetGrid(dataset, col='bias_1', row='confidence_1Z')

In [ ]:
# parallel_coordinates(newd, 'bias')

In [ ]:
fig, axs = plt.subplots(1,3)
for c in range(3):
  h = axs[c].hist2d(dataset['bias_'+str(c+1)], dataset['confidence_'+str(c+1)],cmap='jet', bins=[5,10],density=True)

fig.set_size_inches(25,8)
fig.colorbar(h[3],ax=axs)
plt.show()